<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

## Подготовка данных

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import f1_score, recall_score, roc_auc_score, confusion_matrix, accuracy_score
from sklearn.utils import shuffle
import numpy as np



In [34]:
data = pd.read_csv('/datasets/Churn.csv')
# display(data.head())
print(data.shape)
print(data.info())
print(data.head(20))
print('')
print(data['Exited'].value_counts()) #Чтобы знать, на сколько домножать


(10000, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None
    RowNumber  CustomerId    Surname  CreditScore Geography  Gender  Age  \
0           1    15634602   Hargrave          619    France  Female   42   
1           2    15647311       Hill          608     Spain  Female   41   
2           3    

В столбце Tenure есть пропуски, нужно их обработать.

In [3]:
# Удалил фамилии до преобразования OHE, так как фамилия практической значимости для анализа не несет. Пропущенный значения тоже убрал.
data = data.dropna()
data=data.drop(['Surname'], axis=1)
# print(data.info())

NameError: name 'data' is not defined

In [37]:
#Преобразовал категориальные признаки в численные
data_ohe = pd.get_dummies(data, drop_first=True)


In [38]:
#Разбил данные на 3 выборки
features = data_ohe.drop(['Exited','RowNumber', 'CustomerId'], axis=1)
target = data_ohe['Exited']

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                             target_valid, test_size=0.5, random_state=12345)

In [39]:
# Масштабировал данные
a = features_train # Здесь хранятся исходники, чтобы из объекта numpy преобразовать в датафрейм
b = features_valid
c = features_test

scaler = StandardScaler()
scaler.fit(features_train) 

features_train_scaled = scaler.transform(features_train)
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_train = pd.DataFrame(features_train, index=a.index, columns=a.columns)

features_valid = scaler.transform(features_valid) 
features_valid = pd.DataFrame(features_valid, index=b.index, columns=b.columns)

features_test = scaler.transform(features_test) 
features_test = pd.DataFrame(features_test, index=c.index, columns=c.columns)



pd.options.mode.chained_assignment = None

In [40]:
# Проверил размеры выборок
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)
print(target_train.shape)
print(target_valid.shape)
print(target_test.shape)

(5454, 11)
(1818, 11)
(1819, 11)
(5454,)
(1818,)
(1819,)


In [41]:
features.head(3)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,0,0,0


## Исследование задачи

In [42]:
#DECISION TREE CLASSIFIER
model_dtc = DecisionTreeClassifier (random_state=12345)
model_dtc.fit(features_train, target_train)
predicted_dtc = model_dtc.predict(features_valid)
f1_dtc = f1_score(target_valid, predicted_dtc)
print(f1_dtc)

0.4926764314247669


In [43]:
#LOGISTIC REGRESSION
model_lr = LogisticRegression(random_state=12345, solver='liblinear')
model_lr.fit(features_train, target_train)
predicted_lr = model_lr.predict(features_valid)
f1_lr = f1_score(target_valid, predicted_lr)
print(f1_lr)

0.3033932135728543


In [44]:
#RANDOM FOREST CLASSIFIER
model_rfc = RandomForestClassifier(n_estimators=60, random_state=12345)
model_rfc.fit(features_train, target_train)
predicted_rfc = model_rfc.predict(features_valid)
f1_rfc = f1_score(target_valid, predicted_rfc)
print(f1_rfc)

0.5845648604269295


Без учета баланса классов, модели являются очень неточными, но лучше показывает себя дерево решений

## Борьба с дисбалансом

In [45]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(
    features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [46]:
# Decision Tree Classifier
model_dtc = DecisionTreeClassifier (random_state=12345, class_weight='balanced')
model_dtc.fit(features_upsampled, target_upsampled)
predicted_dtc = model_dtc.predict(features_valid)
f1_dtc = f1_score(target_valid, predicted_dtc)
print(f1_dtc)

0.44925575101488496


In [47]:
# LOGISTIC REGRESSION
model_lr = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model_lr.fit(features_upsampled, target_upsampled)
predicted_lr = model_lr.predict(features_valid)
f1_lr = f1_score(target_valid, predicted_lr)
print(f1_lr)

0.509731232622799


In [48]:
# RANDOM FOREST CLASSIFIER
best_model = None
best_depth = 0
best_est = 0
best_f1 = 0
for est in range(1, 200, 10):
    for depth in range (1, 60, 2):
        model_rfc = RandomForestClassifier(n_estimators=est, random_state=12345, class_weight='balanced', max_depth=depth )
        model_rfc.fit(features_upsampled, target_upsampled)
        predicted_rfc = model_rfc.predict(features_valid)
        f1_rfc = f1_score(target_valid, predicted_rfc)
        if f1_rfc > best_f1:
            best_est = est
            best_depth = depth
            best_f1 = f1_rfc
            best_model = model_rfc
            
print(best_f1, best_est, best_depth)

0.638700947225981 141 13


Баланс классов увеличивает показатель  F-1 на валидационной выборке. Наиболее эффективной моделью является случайный лес, у него метрика 0.63.

## Тестирование модели

In [50]:
print(f1_score(target_test, model_dtc.predict(features_test)))
print(f1_score(target_test, model_lr.predict(features_test)))
print(f1_score(target_test, model_rfc.predict(features_test)))


0.46814404432132967
0.5030549898167006
0.6118721461187213


In [ ]:
#ROC_AUC
probabilities_test_lr = model_lr.predict_proba(features_test)
probabilities_lr_ones = probabilities_test_lr[:, 1]
roc_auc_lr = roc_auc_score(target_test, probabilities_lr_ones)
print(roc_auc_lr)

In [ ]:
def bootstrap (target_valid, predicted, target, count):
 values = []
for i in range(1000):
        target_subsample = target_valid.sample(n=500, random_state=12345, replace=True)
        predicted_subsample = predicted[target_subsample.index]
        values.append(max(target, predicted, 200))
        
        values = pd.Series(values)
    return values.mean()
# return values


print(bootstrap(target_valid_0, predicted_0, target_0, 200))

Метрика F-1 на тестовой выборке = 0.61, это означает что модель досточно точна в определении класса, при этом присваивание класса не происходит слишком часто. ROC_AUC показывает на сколько модель случайного леса (0.78) отличается от случайной модели (0.5), значит наша модель более чем в полтора раза эффективнее случайной модели.
